# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

Import the open-source software libraries

In [12]:
import pandas as pd
import numpy as np
import requests 
#executes http requests i.e. the API calls to the IE x Cloud API to receive stock market data
import xlsxwriter
# save format documents, excel documents from python script
import math

## Importing Our List of Stocks

Importing the constituents of the S&P 500
These constituents change over time, so in an ideal world, we would connect directly to an index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is difficult and not ideal at the moment. 

In [13]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


There are 505 stocks listed in the S&P500 dataframe mentioned above which means that there are 5 stocks that have dual share sturcture in the S&P at the time.

## Acquiring an API Token

IEX Cloud API token is the data provider that shall be used to pull the stock data.

In [14]:
from api import IEX_CLOUD_API_TOKEN

##  API Call

structure our API calls to IEX cloud:

* Market capitalization for each stock
* Price of each stock

IEX Cloud Legacy provides all accounts a free, unlimited use sandbox for testing. Every account will be assigned two test tokens available via the Console. All sandbox endpoints function the same as production, so you will only need to change the base url and token.

From the IEX Cloud Legacy documentation, we require an endpoint that provides us with BOTH Market capitalization for each stock
and Price of each stock. From scanning through the documentation, the endpoint being used will the 'Quote' endpoint.

**Information about the Quote Endpoint**
Data Timing: Real-time 15min delayed End of day

Data Schedule: 4:30am-8pm ET Mon-Fri

Data Source(s): `IEX Cloud` `Investors Exchange` `Consolidated Tape`

All response attributes related to 15 minute delayed market-wide price data are only available to paid plans

In [15]:
symbol = 'AAPL'
#inserting base url for http requests to IEx Cloud API with quote endpoint
api_url =  f'https://sandbox.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
#execute an http request and store the result of that http request in some outside variable
#the object should be in a format that is accessible => json 
data = requests.get(api_url).json()
print(data)

https://sandbox.iex.cloud/v1/data/core/quote/AAPL?token=pk_30859fbb27c74d8388a5e0eafd342026
[{'avgTotalVolume': 56239836, 'calculationPrice': 'close', 'change': -4.74, 'changePercent': -0.02617, 'close': 176.38, 'closeSource': 'official', 'closeTime': 1692907200211, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 176.32, 'delayedPriceTime': 1692907199880, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 176.38, 'extendedPriceTime': 1692921596362, 'high': 181.104, 'highSource': '15 minute delayed price', 'highTime': 1692907199994, 'iexAskPrice': 176.13, 'iexAskSize': 100, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 176.32, 'iexCloseTime': 1692907199899, 'iexLastUpdated': 1692907199899, 'iexMarketPercent': 0.016568218738037075, 'iexOpen': 180.7, 'iexOpenTime': 1692883800152, 'iexRealtimePrice': 176.32, 'iexRealtimeSize': 4, 'iexVolume': 910354, 'lastTradeTime': 1692907199899, 'latestPrice': 176.38, 'latestSource': 'Close', 'latestTime': 'August 24, 2023

# Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [18]:
price = data['currency']

TypeError: list indices must be integers or slices, not str

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: